In [105]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse, unquote

# Limpieza de datos

Los datos procedentes del scraping tienen formatos que no nos interesan (por ejemplo el símbolo euro después de cada precio o 'kms' después de cada distancia). Además hay bastantes NaN que tenemos que tratar. Por eso, antes de pasar a las fase de análisis y de machine learning será necesario que limpiemos los datos.

In [106]:
df=pd.read_csv("data\\raw\\vehiculos_usados.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18199 entries, 0 to 18198
Data columns (total 52 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            18199 non-null  int64  
 1   Fecha extrac                          18199 non-null  object 
 2   Enlace                                18199 non-null  object 
 3   Marca                                 18199 non-null  object 
 4   Modelo                                17712 non-null  object 
 5   Precio                                18199 non-null  object 
 6   Localización                          18199 non-null  object 
 7   Potencia                              18199 non-null  object 
 8   Tipo vendedor                         18199 non-null  object 
 9   Categoría                             18197 non-null  object 
 10  Tipo de vehículo                      18197 non-null  object 
 11  puertas        

C:\Users\mabi1\AppData\Local\Temp\ipykernel_12624\221663255.py:1: DtypeWarning: Columns (37,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("data\\raw\\vehiculos_usados.csv")


In [107]:
df.isna().sum()

Unnamed: 0                                  0
Fecha extrac                                0
Enlace                                      0
Marca                                       0
Modelo                                    487
Precio                                      0
Localización                                0
Potencia                                    0
Tipo vendedor                               0
Categoría                                   2
Tipo de vehículo                            2
puertas                                   114
Versión del país                          726
Núm. de oferta                            726
Garantía                                 3511
Kilometraje                               885
Año                                       885
Tipo de cambio                            131
Capacidad                                2266
Otras fuentes de energía                16685
Consumo de combustible\n2                3204
Color exterior                    

Como podemos ver hay columnas que donde la mayoría de los registros son NaN, en concreto las últimas 23 columnas. Así que empezaremos eliminando esas.

In [108]:
df = df.iloc[:,:-23]
df.isna().sum()

Unnamed: 0                       0
Fecha extrac                     0
Enlace                           0
Marca                            0
Modelo                         487
Precio                           0
Localización                     0
Potencia                         0
Tipo vendedor                    0
Categoría                        2
Tipo de vehículo                 2
puertas                        114
Versión del país               726
Núm. de oferta                 726
Garantía                      3511
Kilometraje                    885
Año                            885
Tipo de cambio                 131
Capacidad                     2266
Otras fuentes de energía     16685
Consumo de combustible\n2     3204
Color exterior                2112
Color original                6046
Tracción                      4175
plazas                        3485
Número de marchas             3835
Número de cilindros           4385
Peso                          5239
Tipo de combustible 

Nos sigue quedando el atributo 'Otras fuentes de energía' con la mayoría de valores en NaN. Lo eliminamos también.

In [109]:
df=df.drop(['Otras fuentes de energía'], axis=1)
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior               2112
Color original               6046
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros          4385
Peso                         5239
Tipo de combustible          1563
dtype: int64

Vamos a ocuparnos ahora de los NaN de los atributos que más tienen a los que menos. Empezaremos con 'Color original' y 'Color exterior'

En fases posteriores de análisis se ha observado que hay valores en esta variable que no corresponden a colores. Los reemplazaremos por None antes de empezar, para intentar arreglarlos.

In [230]:
df.loc[df['Color exterior'] == '.', 'Color exterior'] = None
df.loc[df['Color original'] == '.', 'Color original'] = None
df.loc[df['Color original'] == '4X4', 'Color original'] = None
df.loc[df['Color original'] == '7 PLAZAS', 'Color original'] = None
df.loc[df['Color original'] == 'ACEPTO VEHÍCULO A CAMBIO', 'Color original'] = None
df.loc[df['Color original'] == 'GRAND SPORT ** Q4 350CV', 'Color original'] = None

In [231]:
df.loc[:50, ['Color exterior', 'Color original']]

,Color exterior,Color original
0,Negro,Negro Midnight (metalizado)
1,Gris,Gris
2,Plateado,Gris Plata
3,Gris,Gris Ágata
4,Rojo,Rosso Corsa
5,Negro,Negro
6,Blanco,Blanco
7,Azul,PACK AMG
8,NEGRO OBSIDIANA,NEGRO OBSIDIANA
9,Negro,Negro


Asumiremos que la mayoría de coches no son repintados para cambiarles el color. De manera que cuando haya un NaN en el color original le copiaremos el valor del color exterior y viceversa.

In [232]:
df['Color exterior'].fillna(df['Color original'], inplace=True)
df['Color original'].fillna(df['Color exterior'], inplace=True)

In [233]:
df.isna().sum()

Fecha extrac              0
Enlace                    0
Marca                     0
Modelo                    0
Precio                    0
Localización              0
Potencia                  0
Tipo vendedor             0
Categoría                 0
Tipo de vehículo          0
puertas                   0
Versión del país          0
Garantía                  0
Kilometraje               0
Año                       0
Tipo de cambio            0
Capacidad                 0
Consumo de combustible    0
Color exterior            0
Color original            0
Tracción                  0
plazas                    0
Número de marchas         0
Número de cilindros       0
Peso                      0
Tipo de combustible       0
Mes                       0
Ciudad                    0
CP                        0
provincia                 0
dtype: int64

Con esta maniobra hemos reducido los NaN de la variables 'Color exterior' y 'Color original' considerablemente. No creemos que el color sea un dato tan relevante como para justificar eliminar más de 1000 registros, así que lo rellenaremos con la moda.

In [234]:
print("La moda de 'Color exterior' es", df['Color exterior'].mode()[0])
print("La moda de 'Color original' es", df['Color original'].mode()[0])

La moda de 'Color exterior' es Blanco
La moda de 'Color original' es Blanco


In [235]:
df['Color exterior'].fillna(df['Color exterior'].mode()[0], inplace=True)
df['Color original'].fillna(df['Color original'].mode()[0], inplace=True)
df.isna().sum()

Fecha extrac              0
Enlace                    0
Marca                     0
Modelo                    0
Precio                    0
Localización              0
Potencia                  0
Tipo vendedor             0
Categoría                 0
Tipo de vehículo          0
puertas                   0
Versión del país          0
Garantía                  0
Kilometraje               0
Año                       0
Tipo de cambio            0
Capacidad                 0
Consumo de combustible    0
Color exterior            0
Color original            0
Tracción                  0
plazas                    0
Número de marchas         0
Número de cilindros       0
Peso                      0
Tipo de combustible       0
Mes                       0
Ciudad                    0
CP                        0
provincia                 0
dtype: int64

Ahora vamos a ocuparnos de los NaN del atributo Peso. Primero intentaremos ver si los podemos rellenar cogiendo el peso de los coches de la misma marca y modelo que lo tengan indicado.

In [116]:
df['Peso'] = df.groupby(['Marca', 'Modelo'])['Peso'].fillna(method='ffill')

In [117]:
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros          4385
Peso                         2003
Tipo de combustible          1563
dtype: int64

Así hemos conseguido completar bastantes de los valores NaN. Vamos a ver si completando de atrás hacia alante conseguimos completar alguno más.

In [118]:
df['Peso'] = df.groupby(['Marca', 'Modelo'])['Peso'].fillna(method='bfill')
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros          4385
Peso                          890
Tipo de combustible          1563
dtype: int64

Ahora ya no hay más coches de la misma marca y modelo con los que completar la información. Para los restantes queremos ponerle la media de peso de los coches de esa misma marca. Pero no podemos calcularla aún porque el peso es un texto con el sufijo 'kg'. Primero tendremos que quitarle el sufijo y cambiarle el tipo al atributo 'Peso'

In [119]:
def process_numbers_and_nans(text):
    if pd.isna(text):
        return None
    numbers = re.findall(r'\d+', text)
    return int(''.join(numbers))

In [120]:
df['Peso']=df['Peso'].apply(process_numbers_and_nans)
df['Peso'].head()

0    1439.0
1     950.0
2    1615.0
3    1785.0
4    1290.0
Name: Peso, dtype: float64

Ahora vamos a aplicar la media de la marca a los pesos que siguen en Nan

In [121]:
df['Peso'] = df.groupby(['Marca'])['Peso'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros          4385
Peso                            9
Tipo de combustible          1563
dtype: int64

Los que quedan en Nan deben ser los que no tienen otros vehículos con la misma marca con peso asignado. Así que los rellenaremos con la media de peso de coches, sin filtrar por marca

In [122]:
df['Peso'] = df['Peso'].transform(lambda x: x.fillna(x.mean()))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros          4385
Peso                            0
Tipo de combustible          1563
dtype: int64

Repetiremos el proceso para el número de cilindros. Primero intentaremos llenar hacia alante y hacia atrás con los coches de la misma marca y modelo.

In [123]:
df['Número de cilindros'] = df.groupby(['Marca', 'Modelo'])['Número de cilindros'].fillna(method='ffill')
df['Número de cilindros'] = df.groupby(['Marca', 'Modelo'])['Número de cilindros'].fillna(method='bfill')
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros           619
Peso                            0
Tipo de combustible          1563
dtype: int64

Hemos reducido significativamente el número de NaN. Ahora aplicaremos la moda para esa marca a los que no hemos podido rellenar.

In [124]:
#Si la moda de la Marca es NaN crashea, lo tratamos en try/except
try:
    df['Número de cilindros'] = df.groupby(['Marca'])['Número de cilindros'].transform(lambda x: x.fillna((x.mode()[0])))
except:
    pass
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros           619
Peso                            0
Tipo de combustible          1563
dtype: int64

Y por último los que faltan les asignaremos la media de cilindros de todos nuestros registros.

In [125]:
df['Número de cilindros'] = df['Número de cilindros'].transform(lambda x: x.fillna((x.mode()[0])))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas            3835
Número de cilindros             0
Peso                            0
Tipo de combustible          1563
dtype: int64

Como no puede haber decimales en el número de cilindros que contiene un coche cambiaremos el tipo del atributo a entero.

In [126]:
df['Número de cilindros']=df['Número de cilindros'].astype('int64')

Repetiremos el mismo proceso con el número de marchas en el mismo orden. Rellenado con valores de la misma marca y modelo hacia alante y hacia atrás, media de la marca y media de de los coches

In [127]:
df['Número de marchas'] = df.groupby(['Marca', 'Modelo'])['Número de marchas'].fillna(method='ffill')
df['Número de marchas'] = df.groupby(['Marca', 'Modelo'])['Número de marchas'].fillna(method='bfill')
#Cuando la moda de la marca para ese atributo es NaN crashea, lo gestionamos con try/except
try:
    df['Número de marchas'] = df.groupby(['Marca'])['Número de marchas'].transform(lambda x: x.fillna((x.mode()[0])))
except:
    pass
df['Número de marchas'] = df['Número de marchas'].transform(lambda x: x.fillna((x.mode()[0])))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                     4175
plazas                       3485
Número de marchas               0
Número de cilindros             0
Peso                            0
Tipo de combustible          1563
dtype: int64

Cambiaremos la columna a entera porque no puede haber número de marchas decimales

In [128]:
df['Número de marchas']=df['Número de marchas'].astype('int64')
df['Número de marchas'].value_counts()

Número de marchas
6     7416
5     2959
8     2866
7     2366
1     1622
9      593
4      262
2       53
3       39
10      23
Name: count, dtype: int64

Mismo proceso para el atributo 'Tracción' en este caso no se trata de un atributo numérico.

In [129]:
df['Tracción'] = df.groupby(['Marca', 'Modelo'])['Tracción'].fillna(method='ffill')
df['Tracción'] = df.groupby(['Marca', 'Modelo'])['Tracción'].fillna(method='bfill')
#El siguiente bloque está con try/except porque si la moda de la marca era NaN crasheaba.
try:
    df['Tracción'] = df.groupby(['Marca'])['Tracción'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['Tracción'] = df['Tracción'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                     3511
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                        0
plazas                       3485
Número de marchas               0
Número de cilindros             0
Peso                            0
Tipo de combustible          1563
dtype: int64

Pasamos al atributo 'Garantía'. Lo primero que haremos es quitar el sufijo 'mes' que aparece después de cada valor.

In [130]:
df['Garantía']=df['Garantía'].str.extract(r'(\d+)')
df['Garantía'].head(20)

0      12
1      12
2      12
3      12
4      12
5      18
6      12
7      12
8      12
9      12
10     24
11     12
12     24
13    NaN
14    NaN
15     12
16     36
17    NaN
18     12
19     12
Name: Garantía, dtype: object

En España la legislación dice que los vendedores particulares deben ofrecer 6 meses de garantía para cubrir las averías ocultas que un vehículo de segunda mano pudiera tener.
En el caso de vendedores profesionales la garantía que deben ofrecer es de 12 meses. Así que rellenaremos los Nan según la legalidad que ha de ofrecer el tipo de vendedor ya que asumiremos que si no han especificado que ofrecen mayor garantía es que ofrecen la mínima legal.

In [131]:
df['Garantía'] = df.apply(lambda row: 6 if row['Tipo vendedor'] != 'Prof.' and pd.isna(row['Garantía']) else row['Garantía'], axis=1)
df['Garantía'].fillna(12, inplace=True)
df.isna().sum()

Unnamed: 0                      0
Fecha extrac                    0
Enlace                          0
Marca                           0
Modelo                        487
Precio                          0
Localización                    0
Potencia                        0
Tipo vendedor                   0
Categoría                       2
Tipo de vehículo                2
puertas                       114
Versión del país              726
Núm. de oferta                726
Garantía                        0
Kilometraje                   885
Año                           885
Tipo de cambio                131
Capacidad                    2266
Consumo de combustible\n2    3204
Color exterior                  0
Color original                  0
Tracción                        0
plazas                       3485
Número de marchas               0
Número de cilindros             0
Peso                            0
Tipo de combustible          1563
dtype: int64

El atributo 'Consumo de combustible' contiene en realidad 3 datos (consumo mixto, urbano y extraurbano). Nos quedaremos solo con el consumo mixto y únicamente con la cifra

In [132]:
df=df.rename(columns={'Consumo de combustible\n2' : 'Consumo de combustible'})

In [133]:
df['Consumo de combustible'][0]

'6,6 l/100 km (mixto)'

In [134]:
df['Consumo de combustible'] = df['Consumo de combustible'].str.extract(r'(\d+,\d+)')

In [135]:
df['Consumo de combustible'].head()

0     6,6
1     5,4
2     NaN
3     6,2
4    13,6
Name: Consumo de combustible, dtype: object

Vamos a ver cuántos NaN tenemos y a limpiarlos con el proceso que hemos seguido en la mayoría de campos.

In [136]:
df['Consumo de combustible'].isna().sum()

6692

In [137]:
df['Consumo de combustible']=df['Consumo de combustible'].str.replace(',', '.')
df['Consumo de combustible']=df['Consumo de combustible'].astype('float64')

In [138]:
df['Consumo de combustible'] = df.groupby(['Marca', 'Modelo'])['Consumo de combustible'].fillna(method='ffill')
df['Consumo de combustible'] = df.groupby(['Marca', 'Modelo'])['Consumo de combustible'].fillna(method='bfill')
try:
    df['Consumo de combustible'] = df.groupby(['Marca'])['Consumo de combustible'].transform(lambda x: x.fillna(x.mean()))
except:
    pass
df['Consumo de combustible'] = df['Consumo de combustible'].transform(lambda x: x.fillna(x.mean()))
df['Consumo de combustible'].isna().sum()

0

A continuación nos ocuparemos de la capacidad. Empezaremos quitando 'cm3' después de los valores y dejando solo el número, como hicimos con los kms. Luego llenaremos según marca y modelo, luego la media de la marca, y finalmente la media total

In [139]:
df['Capacidad']=df['Capacidad'].apply(process_numbers_and_nans)

In [140]:
df['Capacidad'] = df.groupby(['Marca', 'Modelo'])['Capacidad'].fillna(method='ffill')
df['Capacidad'] = df.groupby(['Marca', 'Modelo'])['Capacidad'].fillna(method='bfill')
df['Capacidad'] = df.groupby(['Marca'])['Capacidad'].transform(lambda x: x.fillna(x.mean()))
df['Capacidad'] = df['Capacidad'].transform(lambda x: x.fillna(x.mean()))

In [141]:
df.isna().sum()

Unnamed: 0                   0
Fecha extrac                 0
Enlace                       0
Marca                        0
Modelo                     487
Precio                       0
Localización                 0
Potencia                     0
Tipo vendedor                0
Categoría                    2
Tipo de vehículo             2
puertas                    114
Versión del país           726
Núm. de oferta             726
Garantía                     0
Kilometraje                885
Año                        885
Tipo de cambio             131
Capacidad                    0
Consumo de combustible       0
Color exterior               0
Color original               0
Tracción                     0
plazas                    3485
Número de marchas            0
Número de cilindros          0
Peso                         0
Tipo de combustible       1563
dtype: int64

El mismo proceso que venimos aplicando ahora para el número de plazas.

In [142]:
df['plazas'] = df.groupby(['Marca', 'Modelo'])['plazas'].fillna(method='ffill')
df['plazas'] = df.groupby(['Marca', 'Modelo'])['plazas'].fillna(method='bfill')
try:
    df['plazas'] = df.groupby(['Marca'])['plazas'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['plazas'] = df['plazas'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                   0
Fecha extrac                 0
Enlace                       0
Marca                        0
Modelo                     487
Precio                       0
Localización                 0
Potencia                     0
Tipo vendedor                0
Categoría                    2
Tipo de vehículo             2
puertas                    114
Versión del país           726
Núm. de oferta             726
Garantía                     0
Kilometraje                885
Año                        885
Tipo de cambio             131
Capacidad                    0
Consumo de combustible       0
Color exterior               0
Color original               0
Tracción                     0
plazas                       0
Número de marchas            0
Número de cilindros          0
Peso                         0
Tipo de combustible       1563
dtype: int64

Es el momento del 'Tipo de combustible'. El tipo de combustible afecta significativamente al precio, que será nuestra variable target en ML. Los vehículos gasolina son más baratos que los diesel y para una misma marca y modelo muchas veces existe la versión gasolina y la versión diesel. Por lo tanto no parece razonable realizar el proceso de completar que venimos haciendo en otros atributos, sino más bien eliminar los registros que no tienen especificado el tipo de combustible.

In [143]:
df=df[df['Tipo de combustible'].notna()]
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    464
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    91
Versión del país          682
Núm. de oferta            682
Garantía                    0
Kilometraje               711
Año                       711
Tipo de cambio            113
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                      0
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
dtype: int64

Vamos a separar el atributo 'Año' que viene en formato mm/aaaa en una variable 'Año' y otra 'Mes'

In [144]:
df['Año'].value_counts()

Año
04/2023    317
06/2019    255
07/2019    246
07/2022    240
01/2019    239
          ... 
05/1929      1
02/1996      1
08/2001      1
10/1968      1
04/1969      1
Name: count, Length: 610, dtype: int64

In [145]:
df['Año'].fillna('', inplace=True)
df[['Mes', 'Año']] = df['Año'].str.split('/', expand=True)
df['Año']=df['Año'].apply(process_numbers_and_nans)

El año es otro atributo importante para el precio. Y dificil de imputar pues hay modelos de vehículos que llevan fabricándose más de una década y marcas que llevan muchas décadas en funcionamiento. Así que en este caso también optaremos por eliminar las filas que no tienen especificado el año.

In [146]:
df=df[df['Año'].notna()]
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    440
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    91
Versión del país          682
Núm. de oferta            682
Garantía                    0
Kilometraje                 0
Año                         0
Tipo de cambio            113
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                      0
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

Pasamos al atributo 'Version del país'. El único valor disponible en el dataset es España, así que es el que asignaremos a todos los vehículos con Nan en ese atributo

In [147]:
df['Versión del país'].value_counts()

Versión del país
España    15243
Name: count, dtype: int64

In [148]:
df['Versión del país'].fillna('España', inplace=True)
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    440
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    91
Versión del país            0
Núm. de oferta            682
Garantía                    0
Kilometraje                 0
Año                         0
Tipo de cambio            113
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                      0
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

El siguiente atributo con NaNs es 'Núm. de oferta'. Este atributo al ser un identificador nos sirve para detectar duplicados, pero una vez se han eliminado estos no tiene interés para el análisis ni el modelo, así que lo eliminaremos.

In [149]:
df['Núm. de oferta'].value_counts()

Núm. de oferta
10794610.0    3
11062646.0    2
7843231.0     2
10961456.0    2
10889853.0    2
             ..
11081802.0    1
11313733.0    1
10842500.0    1
8117767.0     1
11467880.0    1
Name: count, Length: 15211, dtype: int64

In [150]:
df[df['Núm. de oferta']==10794610.0]

,Unnamed: 0,Fecha extrac,Enlace,Marca,Modelo,Precio,Localización,Potencia,Tipo vendedor,Categoría,...,Consumo de combustible,Color exterior,Color original,Tracción,plazas,Número de marchas,Número de cilindros,Peso,Tipo de combustible,Mes
488,488,2023-11-11,https://www.autoscout24.es/anuncios/mitsubishi...,Mitsubishi,Montero,"€ 5.900,-",https://maps.google.com/?q=POL%C3%8DGONO%20B%2...,92 kW (125 CV),Prof.,SUV/4x4/Pickup,...,5.7,Azul,Azul,Tracción delantera,4.0,6,4,1220.0,Diésel,04
9124,9124,2023-11-19,https://www.autoscout24.es/anuncios/mitsubishi...,Mitsubishi,Montero,"€ 5.900,-",https://maps.google.com/?q=POL%C3%8DGONO%20B%2...,92 kW (125 CV),Prof.,SUV/4x4/Pickup,...,4.8,Azul,Azul,Tracción delantera,5.0,6,4,1581.0,Diésel,04
13725,13725,2023-11-23,https://www.autoscout24.es/anuncios/mitsubishi...,Mitsubishi,Montero,"€ 5.900,-",https://maps.google.com/?q=POL%C3%8DGONO%20B%2...,92 kW (125 CV),Prof.,SUV/4x4/Pickup,...,6.4,Azul,Azul,Tracción delantera,4.0,7,4,1225.0,Diésel,04


Aunque hay alguna discrepancia entre los anuncios, parece el mismo vehículo que ha sido anunciado varias veces. Eliminaremos los duplicados y nos quedaremos con la última versión, que es la que debe tener las correcciones y el precio más actualizado.

In [151]:
df=df.drop_duplicates(['Núm. de oferta'], keep='last')

In [152]:
df['Núm. de oferta'].value_counts()

Núm. de oferta
9718315.0     1
11171146.0    1
9593462.0     1
10999755.0    1
11160671.0    1
             ..
10711111.0    1
11082085.0    1
11082208.0    1
11107759.0    1
11467880.0    1
Name: count, Length: 15211, dtype: int64

In [153]:
df=df.drop(['Núm. de oferta'], axis=1)
df.isna().sum()

Unnamed: 0                  0
Fecha extrac                0
Enlace                      0
Marca                       0
Modelo                    402
Precio                      0
Localización                0
Potencia                    0
Tipo vendedor               0
Categoría                   0
Tipo de vehículo            0
puertas                    14
Versión del país            0
Garantía                    0
Kilometraje                 0
Año                         0
Tipo de cambio              5
Capacidad                   0
Consumo de combustible      0
Color exterior              0
Color original              0
Tracción                    0
plazas                      0
Número de marchas           0
Número de cilindros         0
Peso                        0
Tipo de combustible         0
Mes                         0
dtype: int64

Los NaN en el atributo 'Modelo' son porque no se ha especificado el modelo en la casilla correspondiente al poner el anuncio. A veces se puede encontrar en la descripción, pero dado que son pocos no compensa el trabajo de intentar encontrarlo allí. Puesto que el modelo es un atributo muy importante a la hora de calcular el precio, tampoco parece razonable imputarlo a través de la moda o similares. Así que borraremos los registros que no tienen el modelo especificado.

In [154]:
df=df[df['Modelo'].notna()]
df.isna().sum()

Unnamed: 0                 0
Fecha extrac               0
Enlace                     0
Marca                      0
Modelo                     0
Precio                     0
Localización               0
Potencia                   0
Tipo vendedor              0
Categoría                  0
Tipo de vehículo           0
puertas                   14
Versión del país           0
Garantía                   0
Kilometraje                0
Año                        0
Tipo de cambio             5
Capacidad                  0
Consumo de combustible     0
Color exterior             0
Color original             0
Tracción                   0
plazas                     0
Número de marchas          0
Número de cilindros        0
Peso                       0
Tipo de combustible        0
Mes                        0
dtype: int64

Para los NaN en el atributo 'puertas' usaremos el método de copiarlo de coches de la misma marca y modelo o coger la moda de la marca o general.

In [155]:
df['puertas'] = df.groupby(['Marca', 'Modelo'])['puertas'].fillna(method='ffill')
df['puertas'] = df.groupby(['Marca', 'Modelo'])['puertas'].fillna(method='bfill')
try:
    df['puertas'] = df.groupby(['Marca'])['puertas'].transform(lambda x: x.fillna(x.mode()[0]))
except:
    pass
df['puertas'] = df['puertas'].transform(lambda x: x.fillna(x.mode()[0]))
df.isna().sum()

Unnamed: 0                0
Fecha extrac              0
Enlace                    0
Marca                     0
Modelo                    0
Precio                    0
Localización              0
Potencia                  0
Tipo vendedor             0
Categoría                 0
Tipo de vehículo          0
puertas                   0
Versión del país          0
Garantía                  0
Kilometraje               0
Año                       0
Tipo de cambio            5
Capacidad                 0
Consumo de combustible    0
Color exterior            0
Color original            0
Tracción                  0
plazas                    0
Número de marchas         0
Número de cilindros       0
Peso                      0
Tipo de combustible       0
Mes                       0
dtype: int64

Nos quedan solo unos pocos NaN en el atributo 'Tipo de cambio', eliminaremos los registros, ya que son muy pocos e imposibles de imputar ya que el mismo modelo suele salir en cambio Manual y automático.

In [156]:
df=df[df['Tipo de cambio'].notna()]
df.isna().sum()

Unnamed: 0                0
Fecha extrac              0
Enlace                    0
Marca                     0
Modelo                    0
Precio                    0
Localización              0
Potencia                  0
Tipo vendedor             0
Categoría                 0
Tipo de vehículo          0
puertas                   0
Versión del país          0
Garantía                  0
Kilometraje               0
Año                       0
Tipo de cambio            0
Capacidad                 0
Consumo de combustible    0
Color exterior            0
Color original            0
Tracción                  0
plazas                    0
Número de marchas         0
Número de cilindros       0
Peso                      0
Tipo de combustible       0
Mes                       0
dtype: int64

In [157]:
df.head(10)

,Unnamed: 0,Fecha extrac,Enlace,Marca,Modelo,Precio,Localización,Potencia,Tipo vendedor,Categoría,...,Consumo de combustible,Color exterior,Color original,Tracción,plazas,Número de marchas,Número de cilindros,Peso,Tipo de combustible,Mes
1,1,2023-11-11,https://www.autoscout24.es/anuncios/cupra-form...,Cupra,Formentor,"€ 20.995,-",https://maps.google.com/?q=AVENIDA%20DE%20CATA...,110 kW (150 CV),Prof.,SUV/4x4/Pickup,...,4.8,Negro,Negro Midnight (metalizado),Tracción delantera,4.0,6,4,950.0,Gasolina,05
2,2,2023-11-11,https://www.autoscout24.es/anuncios/gmc-yukon-...,GMC,Yukon,"€ 79.000,-",https://maps.google.com/?q=C%2F%20VICENTE%20MU...,313 kW (426 CV),Prof.,SUV/4x4/Pickup,...,4.1,Gris,Gris,Tracción delantera,5.0,1,4,1615.0,Gasolina,06
4,4,2023-11-11,https://www.autoscout24.es/anuncios/audi-r8-4-...,Audi,R8,"€ 51.500,-",https://maps.google.com/?q=Ctra.%20Madrid%2C%2...,309 kW (420 CV),Prof.,Coupé,...,4.1,Plateado,Gris Plata,Tracción delantera,5.0,5,4,1290.0,Gasolina,06
5,5,2023-11-11,https://www.autoscout24.es/anuncios/porsche-99...,Porsche,992,"€ 289.900,-",https://maps.google.com/?q=Poligono%20Industri...,478 kW (650 CV),Prof.,Coupé,...,7.2,Gris,Gris Ágata,Tracción a las cuatro ruedas,4.0,7,6,1785.0,Gasolina,11
6,6,2023-11-11,https://www.autoscout24.es/anuncios/ferrari-81...,Ferrari,812,"€ 379.999,-",https://maps.google.com/?q=C%2FCARRIL%20DE%20P...,588 kW (799 CV),Prof.,Coupé,...,4.3,Rojo,Rosso Corsa,Tracción trasera,5.0,8,4,1565.0,Gasolina,01
7,7,2023-11-11,https://www.autoscout24.es/anuncios/ds-automob...,DS Automobiles,DS 7 Crossback,"€ 21.290,-1",https://maps.google.com/?q=AV.%20DE%20AR%C3%93...,96 kW (131 CV),Prof.,SUV/4x4/Pickup,...,4.6,Negro,Negro,Tracción delantera,5.0,6,4,1503.0,Diésel,03
8,8,2023-11-11,https://www.autoscout24.es/anuncios/bentley-co...,Bentley,Continental,"€ 64.500,-",https://maps.google.com/?q=Crtra.de%20Cadiz%20...,449 kW (610 CV),Prof.,Coupé,...,4.5,Blanco,Blanco,Tracción delantera,5.0,6,4,1541.0,Gasolina,06
9,9,2023-11-11,https://www.autoscout24.es/anuncios/mercedes-b...,Mercedes-Benz,GLC 250,"€ 35.900,-1",https://maps.google.com/?q=AVDA%20CAMINO%20DE%...,150 kW (204 CV),Prof.,SUV/4x4/Pickup,...,5.5,Azul,PACK AMG,Tracción delantera,5.0,6,3,1365.0,Diésel,05
10,10,2023-11-11,https://www.autoscout24.es/anuncios/mercedes-b...,Mercedes-Benz,GLS 500,"€ 69.991,-",https://maps.google.com/?q=CARRETERA%20DE%20MA...,335 kW (455 CV),Prof.,SUV/4x4/Pickup,...,3.4,NEGRO OBSIDIANA,NEGRO OBSIDIANA,Tracción delantera,4.0,6,3,1115.0,Gasolina,07
11,11,2023-11-11,https://www.autoscout24.es/anuncios/ferrari-f1...,Ferrari,F12,"€ 195.000,-",https://maps.google.com/?q=Avenida%20de%20Somo...,545 kW (741 CV),Prof.,Coupé,...,6.5,Negro,Negro,Tracción a las cuatro ruedas,5.0,9,4,1768.0,Gasolina,02


Ahora que ya hemos tratado los NaN iremos a limpiar los problemas de formato. Lo primero que haremos es eliminar la columna 'Unnamed: 0' que equivale al índice. Y a continuación quitaremos el símbolo de euro y el punto al atributo 'Precio'

In [158]:
df=df.drop(['Unnamed: 0'], axis=1)

In [159]:
df['Precio']=df['Precio'].apply(process_numbers_and_nans)

Del atributo 'Potencia' nos quedaremos con los caballos (CV) ya que es la medida que se usa en España.

In [160]:
df['Potencia']=df['Potencia'].str.extract(r'\((\d+)\s*CV\)')
df['Potencia'].head()

1    150
2    426
4    420
5    650
6    799
Name: Potencia, dtype: object

In [161]:
df['Potencia'].isna().sum()

370

El cambio de formato ha dejado al descubierto los NaN que no se veían antes por tener un valor predefinido de texto. Vamos a copiarle la potencia de los vehículos de la misma marca y modelo. Y los que no se puedan completar así le asignaremos la media de la marca. 

In [162]:
df['Potencia'] = df.groupby(['Marca', 'Modelo'])['Potencia'].fillna(method='ffill')
df['Potencia'] = df.groupby(['Marca', 'Modelo'])['Potencia'].fillna(method='bfill')
df['Potencia'].isna().sum()

147

Eliminaremos los NaN restantes

In [163]:
df=df[df['Potencia'].notna()]
df['Potencia'].isna().sum()

0

Al atributo 'Kilometraje' hay que quitarle el 'km' después del número y el punto de separación de miles.

In [164]:
df['Kilometraje']=df['Kilometraje'].apply(process_numbers_and_nans)
df['Kilometraje'].head()

1    26141
2    81422
4    81100
5     1650
6     4600
Name: Kilometraje, dtype: int64

A continuación vamos a ver qué columnas no están en el formato que nos interesa y se lo cambiaremos

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14658 entries, 1 to 18198
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Fecha extrac            14658 non-null  object 
 1   Enlace                  14658 non-null  object 
 2   Marca                   14658 non-null  object 
 3   Modelo                  14658 non-null  object 
 4   Precio                  14658 non-null  int64  
 5   Localización            14658 non-null  object 
 6   Potencia                14658 non-null  object 
 7   Tipo vendedor           14658 non-null  object 
 8   Categoría               14658 non-null  object 
 9   Tipo de vehículo        14658 non-null  object 
 10  puertas                 14658 non-null  float64
 11  Versión del país        14658 non-null  object 
 12  Garantía                14658 non-null  object 
 13  Kilometraje             14658 non-null  int64  
 14  Año                     14658 non-null  flo

In [166]:
df[['puertas', 'Garantía', 'Año', 'plazas', 'Mes', 'Potencia']]=df[['puertas', 'Garantía', 'Año', 'plazas', 'Mes', 'Potencia']].astype('int64')
df[['Precio', 'Kilometraje']]=df[['Precio', 'Kilometraje']].astype('float64')

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14658 entries, 1 to 18198
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Fecha extrac            14658 non-null  object 
 1   Enlace                  14658 non-null  object 
 2   Marca                   14658 non-null  object 
 3   Modelo                  14658 non-null  object 
 4   Precio                  14658 non-null  float64
 5   Localización            14658 non-null  object 
 6   Potencia                14658 non-null  int64  
 7   Tipo vendedor           14658 non-null  object 
 8   Categoría               14658 non-null  object 
 9   Tipo de vehículo        14658 non-null  object 
 10  puertas                 14658 non-null  int64  
 11  Versión del país        14658 non-null  object 
 12  Garantía                14658 non-null  int64  
 13  Kilometraje             14658 non-null  float64
 14  Año                     14658 non-null  int

In [168]:
df.head()

,Fecha extrac,Enlace,Marca,Modelo,Precio,Localización,Potencia,Tipo vendedor,Categoría,Tipo de vehículo,...,Consumo de combustible,Color exterior,Color original,Tracción,plazas,Número de marchas,Número de cilindros,Peso,Tipo de combustible,Mes
1,2023-11-11,https://www.autoscout24.es/anuncios/cupra-form...,Cupra,Formentor,20995.0,https://maps.google.com/?q=AVENIDA%20DE%20CATA...,150,Prof.,SUV/4x4/Pickup,Ocasión,...,4.8,Negro,Negro Midnight (metalizado),Tracción delantera,4,6,4,950.0,Gasolina,5
2,2023-11-11,https://www.autoscout24.es/anuncios/gmc-yukon-...,GMC,Yukon,79000.0,https://maps.google.com/?q=C%2F%20VICENTE%20MU...,426,Prof.,SUV/4x4/Pickup,Ocasión,...,4.1,Gris,Gris,Tracción delantera,5,1,4,1615.0,Gasolina,6
4,2023-11-11,https://www.autoscout24.es/anuncios/audi-r8-4-...,Audi,R8,51500.0,https://maps.google.com/?q=Ctra.%20Madrid%2C%2...,420,Prof.,Coupé,Ocasión,...,4.1,Plateado,Gris Plata,Tracción delantera,5,5,4,1290.0,Gasolina,6
5,2023-11-11,https://www.autoscout24.es/anuncios/porsche-99...,Porsche,992,289900.0,https://maps.google.com/?q=Poligono%20Industri...,650,Prof.,Coupé,Demostración,...,7.2,Gris,Gris Ágata,Tracción a las cuatro ruedas,4,7,6,1785.0,Gasolina,11
6,2023-11-11,https://www.autoscout24.es/anuncios/ferrari-81...,Ferrari,812,379999.0,https://maps.google.com/?q=C%2FCARRIL%20DE%20P...,799,Prof.,Coupé,Ocasión,...,4.3,Rojo,Rosso Corsa,Tracción trasera,5,8,4,1565.0,Gasolina,1


Sobre la localización, actualmente tenemos un enlace de Google Maps, que nos puede ser muy útil, pero nos gustaría tener también un atributo 'Ciudad'. Esa información está contenida en el enlace de Google Maps, así que vamos a parsearlo para obtenerla, también extraeremos el código postal.

In [169]:
def obtener_ciudad_cp(url):
    parsed_url = urlparse(url)
    query_params = dict(pair.split('=') for pair in parsed_url.query.split('&'))
    
    # Decodificar la dirección de la URL
    decoded_address = unquote(query_params['q'])
    
    # Dividir la dirección en partes
    partes_direccion = decoded_address.split(',')
    
    # Obtener la ciudad y el código postal
    cp_ciudad = partes_direccion[-2].strip()
    #A veces está en una posición distinta
    if not cp_ciudad:
        cp_ciudad=partes_direccion[-3].strip()
    
    #La ciudad contiene actualmente el código postal, vamos a separlo en dos atributos
    cp, ciudad = cp_ciudad.split(' ', 1) if ' ' in cp_ciudad else (cp_ciudad, '')
    #Los códigos postales han de tener 5 carácteres, pero la dirección lo tiene como entero sin los "0" de inicio, se los añadimos
    if len(cp) < 5:
        cp= "0" * (5 - len(cp)) + cp
    ciudad=ciudad.capitalize()

    return ciudad, cp

In [170]:
df[['Ciudad', 'CP']] = df['Localización'].apply(lambda x: pd.Series(obtener_ciudad_cp(x)))
df[['Localización','Ciudad', 'CP']].head(10)

,Localización,Ciudad,CP
1,https://maps.google.com/?q=AVENIDA%20DE%20CATA...,Zaragoza,50014
2,https://maps.google.com/?q=C%2F%20VICENTE%20MU...,Madrid,28043
4,https://maps.google.com/?q=Ctra.%20Madrid%2C%2...,Alicante,03006
5,https://maps.google.com/?q=Poligono%20Industri...,Cardedeu,08440
6,https://maps.google.com/?q=C%2FCARRIL%20DE%20P...,Marbella,29670
7,https://maps.google.com/?q=AV.%20DE%20AR%C3%93...,Pamplona,31009
8,https://maps.google.com/?q=Crtra.de%20Cadiz%20...,San pedro de alcantara,29670
9,https://maps.google.com/?q=AVDA%20CAMINO%20DE%...,San sebastian de los reyes,28703
10,https://maps.google.com/?q=CARRETERA%20DE%20MA...,Jaen,23009
11,https://maps.google.com/?q=Avenida%20de%20Somo...,San sebastian de los reyes,28703


Ya tenemos la ciudad y el código postal, pero nos gustaría conocer también la provincia. La obtendremos con con el dataset de códigos postales 'postalcat.csv' extraido de https://postal.cat/index.es.html

In [171]:
df_cp=pd.read_csv("data\\raw\\postalcat.csv", sep=";", dtype={'cp': str})
df_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156750 entries, 0 to 156749
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   internalid   156750 non-null  int64 
 1   cp           156750 non-null  object
 2   carrer       118688 non-null  object
 3   poblacio     156750 non-null  object
 4   provinciaid  156750 non-null  int64 
 5   provincia    156750 non-null  object
 6   paisid       156750 non-null  object
 7   pais         156750 non-null  object
dtypes: int64(2), object(6)
memory usage: 9.6+ MB


El df_cp tiene múltiples líneas para cada 'cp' porque hay distintas poblaciones aledañas que comparten código postal. Como solo queremos la provincia borraremos los duplicados para que no se dupliquen en el df al hacer el merge.

In [172]:
df_cp=df_cp.drop_duplicates(['cp'], keep='first')
df_cp['cp'].value_counts()

cp
01193    1
37730    1
37712    1
37860    1
37336    1
        ..
22640    1
22415    1
22268    1
22520    1
52006    1
Name: count, Length: 11141, dtype: int64

In [173]:
df=pd.merge(df, df_cp[['cp', 'provincia']], left_on='CP', right_on='cp', how='left')
df=df.drop('cp', axis=1)
df.head()

,Fecha extrac,Enlace,Marca,Modelo,Precio,Localización,Potencia,Tipo vendedor,Categoría,Tipo de vehículo,...,Tracción,plazas,Número de marchas,Número de cilindros,Peso,Tipo de combustible,Mes,Ciudad,CP,provincia
0,2023-11-11,https://www.autoscout24.es/anuncios/cupra-form...,Cupra,Formentor,20995.0,https://maps.google.com/?q=AVENIDA%20DE%20CATA...,150,Prof.,SUV/4x4/Pickup,Ocasión,...,Tracción delantera,4,6,4,950.0,Gasolina,5,Zaragoza,50014,Zaragoza
1,2023-11-11,https://www.autoscout24.es/anuncios/gmc-yukon-...,GMC,Yukon,79000.0,https://maps.google.com/?q=C%2F%20VICENTE%20MU...,426,Prof.,SUV/4x4/Pickup,Ocasión,...,Tracción delantera,5,1,4,1615.0,Gasolina,6,Madrid,28043,Madrid
2,2023-11-11,https://www.autoscout24.es/anuncios/audi-r8-4-...,Audi,R8,51500.0,https://maps.google.com/?q=Ctra.%20Madrid%2C%2...,420,Prof.,Coupé,Ocasión,...,Tracción delantera,5,5,4,1290.0,Gasolina,6,Alicante,03006,Alicante/Alacant
3,2023-11-11,https://www.autoscout24.es/anuncios/porsche-99...,Porsche,992,289900.0,https://maps.google.com/?q=Poligono%20Industri...,650,Prof.,Coupé,Demostración,...,Tracción a las cuatro ruedas,4,7,6,1785.0,Gasolina,11,Cardedeu,08440,Barcelona
4,2023-11-11,https://www.autoscout24.es/anuncios/ferrari-81...,Ferrari,812,379999.0,https://maps.google.com/?q=C%2FCARRIL%20DE%20P...,799,Prof.,Coupé,Ocasión,...,Tracción trasera,5,8,4,1565.0,Gasolina,1,Marbella,29670,Málaga


In [174]:
df['provincia'].isna().sum()

42

In [175]:
df[df['provincia'].isna()]['CP'].value_counts()

CP
08097    13
26302    12
28603     6
42702     5
15407     3
08803     1
46429     1
30314     1
Name: count, dtype: int64

En realidad los 41 NaN corresponden solo a 8 códigos postales. Son pocos, así que lo podemos completar a mano.

In [176]:
df.loc[df['CP'] == '08097', 'provincia'] = 'Barcelona'
df.loc[df['CP'] == '26302', 'provincia'] = 'Málaga'
df.loc[df['CP'] == '28603', 'provincia'] = 'Málaga'
df.loc[df['CP'] == '42702', 'provincia'] = 'Sevilla'
df.loc[df['CP'] == '15407', 'provincia'] = 'A Coruña'
df.loc[df['CP'] == '08803', 'provincia'] = 'Barcelona'
df.loc[df['CP'] == '46429', 'provincia'] = 'Valencia/València'
df.loc[df['CP'] == '30314', 'provincia'] = 'Murcia'

In [177]:
df.isna().sum()

Fecha extrac              0
Enlace                    0
Marca                     0
Modelo                    0
Precio                    0
Localización              0
Potencia                  0
Tipo vendedor             0
Categoría                 0
Tipo de vehículo          0
puertas                   0
Versión del país          0
Garantía                  0
Kilometraje               0
Año                       0
Tipo de cambio            0
Capacidad                 0
Consumo de combustible    0
Color exterior            0
Color original            0
Tracción                  0
plazas                    0
Número de marchas         0
Número de cilindros       0
Peso                      0
Tipo de combustible       0
Mes                       0
Ciudad                    0
CP                        0
provincia                 0
dtype: int64

Ya tenemos el primer dataset limpio, vamos a guardarlo en un archivo diferente para usarlo para el análisis y el modelo.

In [237]:
df.to_csv('data\\clean\\veh_usados.csv')

Vamos a cargar el segundo dataset, correspondiente a datos de Autoscout24 España scrapeados en 2018, extraidos de https://www.kaggle.com/datasets/harturo123/online-adds-of-used-cars/

In [179]:
df_2018=pd.read_csv("data\\raw\\spanish_used_car.csv", sep=";")
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93991 entries, 0 to 93990
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          93991 non-null  int64  
 1   make        93991 non-null  object 
 2   model       93991 non-null  object 
 3   version     93991 non-null  object 
 4   months_old  93717 non-null  float64
 5   power       93504 non-null  float64
 6   sale_type   93979 non-null  object 
 7   num_owners  22660 non-null  float64
 8   gear_type   92710 non-null  object 
 9   fuel_type   93926 non-null  object 
 10  kms         93127 non-null  float64
 11  price       93991 non-null  int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 8.6+ MB


In [180]:
df_2018.isna().sum()

ID                0
make              0
model             0
version           0
months_old      274
power           487
sale_type        12
num_owners    71331
gear_type      1281
fuel_type        65
kms             864
price             0
dtype: int64

Vemos que hay NaNs en este dataset. Lo primero que haremos es eliminar la columna 'num_owners' ya que tiene más de la mitad de los datos en NaN. Además, no es un atributo que tengamos en el dataset principal, así que no nos vale para comparativa, que es para lo que queremos este dataset.

In [181]:
df_2018=df_2018.drop('num_owners', axis=1)

Para el atributo 'power' como hemos hecho en el dataset principal, intentaremos obtener la potencia de coches de la misma marca y mismo modelo. Rellenando tanto hacia adelante como hacia atrás.

In [182]:
df_2018['power'] = df_2018.groupby(['make', 'model'])['power'].fillna(method='ffill')
df_2018['power'] = df_2018.groupby(['make', 'model'])['power'].fillna(method='bfill')
df_2018.isna().sum()

ID               0
make             0
model            0
version          0
months_old     274
power            0
sale_type       12
gear_type     1281
fuel_type       65
kms            864
price            0
dtype: int64

In [183]:
df_2018.head()

,ID,make,model,version,months_old,power,sale_type,gear_type,fuel_type,kms,price
0,97860,Porsche,911,Carrera 4 S Coup�,240.0,210.0,classic,manual,gasoline,202000.0,999999
1,27821,Ford,Mustang,Gt500 Cabrio Vendido,54.0,487.0,used,manual,gasoline,30000.0,685000
2,97801,Porsche,911,3.3 Turbo Targa (Precio A Consultar),358.0,220.0,used,manual,gasoline,56300.0,555555
3,98251,Porsche,911,R Unidad 343 De 991-Iva Deducible,14.0,368.0,used,manual,gasoline,2800.0,470000
4,98250,Porsche,911,Gt2 Rs,3.0,515.0,used,NaN,gasoline,10.0,450000


Una búsqueda en Google de la potencia de algunos de los modelos nos servirá para darnos cuenta que en este dataset se ha guardado la potencia en kilowatios mientras que en nuestro dataset principal la tenemos guardada en caballos. Para transformarla basta con multiplicar todos los valores por el ratio de conversión 1.341

In [184]:
df_2018['power']*=1.341
df_2018.head()

,ID,make,model,version,months_old,power,sale_type,gear_type,fuel_type,kms,price
0,97860,Porsche,911,Carrera 4 S Coup�,240.0,281.610,classic,manual,gasoline,202000.0,999999
1,27821,Ford,Mustang,Gt500 Cabrio Vendido,54.0,653.067,used,manual,gasoline,30000.0,685000
2,97801,Porsche,911,3.3 Turbo Targa (Precio A Consultar),358.0,295.020,used,manual,gasoline,56300.0,555555
3,98251,Porsche,911,R Unidad 343 De 991-Iva Deducible,14.0,493.488,used,manual,gasoline,2800.0,470000
4,98250,Porsche,911,Gt2 Rs,3.0,690.615,used,NaN,gasoline,10.0,450000


Las columnas en las que aún quedan NaNs no son fáciles de imputar (el mismo modelo de coche puede tener cambio automático o manual, combustible diesel o gasolina, y es dificil saber cuantos meses de antigüedad o kms puede tener.) Dado que son una parte muy pequeña del dataset de 100.000 registros, eliminaremos esas filas.

In [185]:
df_2018=df_2018.dropna()
df_2018.isna().sum()

ID            0
make          0
model         0
version       0
months_old    0
power         0
sale_type     0
gear_type     0
fuel_type     0
kms           0
price         0
dtype: int64

Algunas columnas categóricas nos van a dar problemas cuando las queramos comparar con el dataset scrapeado porque en este dataset los valores están en inglés y en el otro en español. Este dataset son muchas menos columnas, así que traduciremos los valores de este al mismo formato que tienen en el principal.

In [186]:
df_2018['sale_type'].value_counts()

sale_type
used          84582
km_0           2602
almost_new     2562
demo           1029
new             699
classic          65
Name: count, dtype: int64

In [187]:
df_2018['sale_type']=df_2018['sale_type'].replace({'used': 'Ocasión', 'km_0': 'KM0', 'almost new': 'Casi nuevo', 'demo': 'Demostración',
                                                   'new': 'Nuevo', 'classic': 'Clásica', 'almost_new' : 'Casi nuevo'})

In [188]:
df_2018['gear_type'].value_counts()

gear_type
manual            66888
automatic         24359
semi-automatic      292
Name: count, dtype: int64

In [189]:
df_2018['gear_type']=df_2018['gear_type'].replace({'manual': 'Manual', 'automatic': 'Automático', 'semi-automatic': 'Semi-automático'})

In [190]:
df_2018['fuel_type'].value_counts()

fuel_type
diesel      70141
gasoline    20826
hybrid        433
electric       92
LPG            43
CNG             3
etanol          1
Name: count, dtype: int64

In [191]:
df['Tipo de combustible'].value_counts()

Tipo de combustible
Gasolina             8135
Diésel               5672
Eléctrico             557
Otros                 188
Gas licuado (GLP)     105
Super 95                1
Name: count, dtype: int64

In [192]:
df_2018['fuel_type']=df_2018['fuel_type'].replace({'diesel': 'Diésel', 'Gasoline': 'Gasolina', 'hybrid': 'Híbrido', 'electric': 'Eléctrico', 
                                                   'LPG' : 'Gas licuado (GLP)', 'etanol': 'Etanol'})

Ya tenemos el dataset listo para guardarlo en archivo

In [193]:
df_2018.to_csv('data\\clean\\2018.csv')

Nos queda un último dataset que limpiar antes de empezar con el análisis.

In [194]:
df_alemania=pd.read_csv('data\\raw\\germany_used_cars_dataset_2023.csv')

In [195]:
df_alemania.isna().sum()

Unnamed: 0                      0
brand                           0
model                           0
color                         166
registration_date               4
year                            0
price_in_euro                   0
power_kw                      134
power_ps                      129
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    26873
fuel_consumption_g_km           0
mileage_in_km                 152
offer_description               1
dtype: int64

Llenaremos la variable color con la moda.

In [196]:
df_alemania['color'].fillna(df_alemania['color'].mode()[0], inplace=True)

Eliminaremos las filas con la fecha en NaN y la separaremos en año y mes

In [197]:
df_alemania['registration_date'].head()

0    10/1995
1    02/1995
2    02/1995
3    07/1995
4    11/1996
Name: registration_date, dtype: object

In [198]:
df_alemania=df_alemania[df_alemania['registration_date'].notna()]

In [199]:
df_alemania[['month', 'year']] = df_alemania['registration_date'].str.split('/', expand=True)
df_alemania[['registration_date', 'month', 'year']].head()

,registration_date,month,year
0,10/1995,10,1995
1,02/1995,02,1995
2,02/1995,02,1995
3,07/1995,07,1995
4,11/1996,11,1996


In [200]:
df_alemania.drop(['registration_date'], axis='columns', inplace=True)
df_alemania.head()

,Unnamed: 0,brand,model,color,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description,month
0,0,alfa-romeo,Alfa Romeo GTV,red,1995,1300,148,201,Manual,Petrol,"10,9 l/100 km",260 g/km,160500.0,2.0 V6 TB,10
1,1,alfa-romeo,Alfa Romeo 164,black,1995,24900,191,260,Manual,Petrol,NaN,- (g/km),190000.0,"Q4 Allrad, 3.2L GTA",02
2,2,alfa-romeo,Alfa Romeo Spider,black,1995,5900,110,150,Unknown,Petrol,NaN,- (g/km),129000.0,ALFA ROME 916,02
3,3,alfa-romeo,Alfa Romeo Spider,black,1995,4900,110,150,Manual,Petrol,"9,5 l/100 km",225 g/km,189500.0,2.0 16V Twin Spark L,07
4,4,alfa-romeo,Alfa Romeo 164,red,1996,17950,132,179,Manual,Petrol,"7,2 l/100 km",- (g/km),96127.0,"3.0i Super V6, absoluter Topzustand !",11


Intentaremos llenar todos los valores de power_ps que podamos transformando los valores de power_kw

In [201]:
df_alemania['power_ps'].info()

<class 'pandas.core.series.Series'>
Index: 251075 entries, 0 to 251078
Series name: power_ps
Non-Null Count   Dtype 
--------------   ----- 
250946 non-null  object
dtypes: object(1)
memory usage: 3.8+ MB


'power_ps' tiene algunos valores no numéricos que pondremos a NaN para poder convertir la variable a numérica

In [202]:
filtro_no_numerico = ~df_alemania['power_ps'].astype(str).str.match(r'^\d+$')
df_alemania.loc[filtro_no_numerico, 'power_ps'] = np.nan
df_alemania['power_ps'] = pd.to_numeric(df_alemania['power_ps'])

In [203]:
df_alemania['power_kw'].info()

<class 'pandas.core.series.Series'>
Index: 251075 entries, 0 to 251078
Series name: power_kw
Non-Null Count   Dtype 
--------------   ----- 
250941 non-null  object
dtypes: object(1)
memory usage: 3.8+ MB


Transformaremos también la variable 'power_kw' para poder hacer conversión

In [204]:
filtro_no_numerico = ~df_alemania['power_kw'].astype(str).str.match(r'^\d+$')
df_alemania.loc[filtro_no_numerico, 'power_kw'] = np.nan
df_alemania['power_kw'] = pd.to_numeric(df_alemania['power_kw'])

In [205]:
df_alemania['power_ps'].fillna(df_alemania['power_kw'] * 1.3596, inplace=True)

In [206]:
df_alemania['power_ps'].isna().sum()

176

Ahora la variable 'power_kw' ya no nos es de más utilidad, así que la eliminaremos

In [207]:
df_alemania=df_alemania.drop('power_kw', axis=1)

Intentaremos llenar algunos Nan con la potencia de coches de la misma marca y modelo

In [208]:
df_alemania['power_ps'] = df_alemania.groupby(['brand', 'model'])['power_ps'].fillna(method='ffill')
df_alemania['power_ps'] = df_alemania.groupby(['brand', 'model'])['power_ps'].fillna(method='bfill')
df_alemania['power_ps'].isna().sum()

1

Solo nos ha faltado un registro por completar, lo eliminaremos

In [209]:
df_alemania=df_alemania[df_alemania['power_ps'].notna()]

Los ps versión alemana de los Horse Power (HP) no són lo mismo que los Caballos de fuerza mecánica (CV). Haremos conversión.

In [210]:
df_alemania['power_ps'].describe()

count    251074.000000
mean        172.146621
std         103.423228
min           1.000000
25%         111.000000
50%         150.000000
75%         190.000000
max        2750.470800
Name: power_ps, dtype: float64

Vemos que hay un par de valores incorrectos, los eliminaremos.

In [211]:
df_alemania['power_cv']=df_alemania['power_ps'] * 0.98632
df_alemania['power_cv']=df_alemania['power_cv'].round(0)

Ahora eliminaremos la columna 'power_ps' que ya no nos sirve

In [212]:
df_alemania=df_alemania.drop('power_ps', axis=1)

Los atributos de fuel consumption uno no tiene ningún NaN (galons/km), mientras que otro, que además és el que nos interesa (l/100km), tiene bastantes. Miraremos de hacer la conversión entre unidades. Según una búsqueda en google, el factor de conversión es 3.785411 y los km se tienen que dividir entre 100. Primero hay que arreglar el formato, coger solo la cifra.

In [213]:
df_alemania.isna().sum()

Unnamed: 0                      0
brand                           0
model                           0
color                           0
year                          193
price_in_euro                   0
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    26872
fuel_consumption_g_km           0
mileage_in_km                 148
offer_description               1
month                           0
power_cv                        0
dtype: int64

In [214]:
df_alemania['fuel_consumption_g_km'].value_counts()

fuel_consumption_g_km
- (g/km)             35808
0 g/km                8533
119 g/km              4813
114 g/km              3882
139 g/km              3389
                     ...  
152,6 g/km               1
488 km Reichweite        1
192,2 g/km               1
138,4 g/km               1
77 km Reichweite         1
Name: count, Length: 1500, dtype: int64

Vemos que en realidad el atributo 'fuel_consumption_g_km' tienen más de 40.000 NaN, solo que están enmascarados por el formato. Primero hay algunos que no corresponden a la medida que debería tener g/km, sino que tiene el Reichweite (Rango), imaginamos que por depósito, dado que cada modelo de coche tiende depósitos de diferente capacidad, este dato no aporta valor, sino que crea confusión. vamos a eliminar esas filas.

In [215]:
df_alemania = df_alemania[~df_alemania['fuel_consumption_g_km'].str.contains('Reichweite')]
df_alemania = df_alemania.reset_index(drop=True)

In [216]:
def process_fuel(value):
    if pd.isna(value):
        return value
    elif '-' in value:
        return None
    else:
        cleaned_value = re.sub(r'\.', '', value)
        match = re.search(r'([\d.,]+)', cleaned_value)
        if match:
            match=float(match.group(1).replace(',', '.'))
            if match > 0.0:
                return match
        return None

In [217]:
df_alemania['fuel_consumption_g_km']=df_alemania['fuel_consumption_g_km'].apply(process_fuel)

In [218]:
df_alemania['fuel_consumption_g_km'].value_counts()

fuel_consumption_g_km
119.0    4813
114.0    3882
139.0    3389
130.0    3368
109.0    3356
         ... 
930.0       1
190.3       1
5.7         1
135.8       1
187.3       1
Name: count, Length: 1037, dtype: int64

Vamos a aplicar la misma transformación a 'fuel_consumption_l_100km'

In [219]:
df_alemania['fuel_consumption_l_100km']= df_alemania['fuel_consumption_l_100km'].apply(process_fuel)

In [220]:
df_alemania['fuel_consumption_l_100km'].value_counts()

fuel_consumption_l_100km
4.9       8168
5.1       7658
5.5       7620
5.9       7521
5.3       7458
          ... 
73.0         1
169.0        1
165.0        1
2006.0       1
22.6         1
Name: count, Length: 312, dtype: int64

In [221]:
df_alemania.isna().sum()

Unnamed: 0                      0
brand                           0
model                           0
color                           0
year                          193
price_in_euro                   0
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    24324
fuel_consumption_g_km       45298
mileage_in_km                 139
offer_description               1
month                           0
power_cv                        0
dtype: int64

Vamos a ver si podemos encontrar valores donde podamos completar el consumo de litros por 100km a través de los galones por km

In [222]:
# Llenar NaN en 'fuel_consumption_l_100km' con la conversión desde 'fuel_consumption_g_km'
df_alemania['fuel_consumption_l_100km'] = df_alemania['fuel_consumption_l_100km'].fillna(df_alemania['fuel_consumption_g_km'] * 3.78541 / 100)

# Redondear a un número específico de decimales si es necesario
df_alemania['fuel_consumption_l_100km'] = df_alemania['fuel_consumption_l_100km'].round(2)

In [223]:
df_alemania.isna().sum()

Unnamed: 0                      0
brand                           0
model                           0
color                           0
year                          193
price_in_euro                   0
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    21886
fuel_consumption_g_km       45298
mileage_in_km                 139
offer_description               1
month                           0
power_cv                        0
dtype: int64

Veremos si podemos completar algún campo más llenándolo con los datos de vehículos de su misma marca y modelo.

In [224]:
df_alemania['fuel_consumption_l_100km'] = df_alemania.groupby(['brand', 'model'])['fuel_consumption_l_100km'].fillna(method='ffill')
df_alemania['fuel_consumption_l_100km'] = df_alemania.groupby(['brand', 'model'])['fuel_consumption_l_100km'].fillna(method='bfill')

In [225]:
df_alemania.isna().sum()

Unnamed: 0                      0
brand                           0
model                           0
color                           0
year                          193
price_in_euro                   0
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km      873
fuel_consumption_g_km       45298
mileage_in_km                 139
offer_description               1
month                           0
power_cv                        0
dtype: int64

Hemos llenado los suficientes registros como para poder eliminar los 800 (de más de 250000) sin preocuparnos. También eliminaremos la columna del consumo por galones, que no nos es relevante para nada más.

In [226]:
df_alemania=df_alemania[df_alemania['fuel_consumption_l_100km'].notna()]
df_alemania=df_alemania.drop('fuel_consumption_g_km', axis='columns')

In [227]:
df_alemania.isna().sum()

Unnamed: 0                    0
brand                         0
model                         0
color                         0
year                        193
price_in_euro                 0
transmission_type             0
fuel_type                     0
fuel_consumption_l_100km      0
mileage_in_km               139
offer_description             1
month                         0
power_cv                      0
dtype: int64

Los atributos que nos quedan con NaN no son en mucha cantidad y son muy importantes para nuestro análisis. Así que eliminaremos los registros con NaN restantes.

In [228]:
df_alemania=df_alemania.dropna()

Ya podemos guardar nuestro dataset limpio en un nuevo archivo

In [229]:
df_alemania.to_csv('data\\clean\\alemania.csv')